In [1]:
#import pyspark
#from pyspark import SparkContext
# from pyspark.sql.window import Window
# from pyspark.sql import functions as F
# from pyspark.sql import types as T

import json
import pandas as pd
import numpy as np

### Read Boarding Data

In [4]:
boarding_data = pd.read_json('/local/tarciso/masters/data/bus_trips/preliminary-exp-sample-data/ticketing-sample/doc1-2017051015.txt', lines=True)

In [5]:
boarding_data.head()

,CODLINHA,CODVEICULO,DATANASCIMENTO,DATAUTILIZACAO,NOMELINHA,NUMEROCARTAO,SEXO
0,000,00070,22/11/58,"09/05/17 11:05:57,000000",OPER S/LINHA,1353891,M
1,542,GA117,23/03/72,"09/05/17 17:25:14,000000",BAIRRO NOVO B,2357837,F
2,000,09053,23/03/72,"09/05/17 17:57:33,000000",OPER S/LINHA,2357837,F
3,000,09053,23/03/72,"09/05/17 17:57:28,000000",OPER S/LINHA,2357837,F
4,021,08046,26/01/72,"09/05/17 20:17:34,000000",INTERB II ANTI H,1937533,F


In [7]:
boardings_total = len(boarding_data)

#### Checking number of boardings on route 000

In [10]:
boardings_000 = len(boarding_data[boarding_data['CODLINHA'] == '000'])

In [11]:
print boardings_000, boardings_total, boardings_000/float(boardings_total)

134629 320292 0.420332071984


#### Looking at 000 boardings in more detail

In [12]:
boardings_000_df = boarding_data[boarding_data['CODLINHA'] == '000']

In [13]:
boardings_000_df.head(20)

,CODLINHA,CODVEICULO,DATANASCIMENTO,DATAUTILIZACAO,NOMELINHA,NUMEROCARTAO,SEXO
0,000,00070,22/11/58,"09/05/17 11:05:57,000000",OPER S/LINHA,1353891,M
2,000,09053,23/03/72,"09/05/17 17:57:33,000000",OPER S/LINHA,2357837,F
3,000,09053,23/03/72,"09/05/17 17:57:28,000000",OPER S/LINHA,2357837,F
13,000,03009,28/08/84,"09/05/17 18:57:22,000000",OPER S/LINHA,3591720,F
15,000,03031,29/12/63,"09/05/17 07:20:19,000000",OPER S/LINHA,1225547,M
17,000,05040,17/01/79,"09/05/17 15:29:11,000000",OPER S/LINHA,2699543,F
20,000,09005,14/09/76,"09/05/17 06:59:45,000000",OPER S/LINHA,2788103,F
22,000,03037,01/06/89,"09/05/17 17:00:00,000000",OPER S/LINHA,2900987,F
25,000,05056,02/07/63,"09/05/17 07:42:22,000000",OPER S/LINHA,1947358,M
26,000,09029,22/02/65,"09/05/17 07:00:15,000000",OPER S/LINHA,3543165,M


### Find boarding stations stop ids for line 000 boarding entries

#### Read Line 000 Codes translation table

In [20]:
line_000_terminals = pd.read_csv('/local/tarciso/masters/data/urbs/line-000-terminals-translation-table.csv', dtype = {'URBS_CODE': str})

In [21]:
line_000_terminals.head()

,LINE,NAME,URBS_CODE,STOP_ID,LAT,LON
0,000,TERMINAL BOQUEIRÃO,00001,14489,-25.516723,-49.230568
1,000,TERMINAL BOQUEIRÃO,00002,14489,-25.516723,-49.230568
2,000,TERMINAL BOQUEIRÃO,00003,14489,-25.516723,-49.230568
3,000,TERMINAL CARMO,00004,14485,-25.501311,-49.237825
4,000,TERMINAL CARMO,00005,14485,-25.501311,-49.237825


#### Match CODVEICULO to URBS_CODE to find line 000 boarding stations

In [46]:
matched_000_stations = boardings_000_df.merge(line_000_terminals, left_on='CODVEICULO', right_on='URBS_CODE', how='inner') \
                        .sort_values(by='DATAUTILIZACAO')

In [60]:
matched_000_stations.head(100)

,CODLINHA,CODVEICULO,DATANASCIMENTO,DATAUTILIZACAO,NOMELINHA,NUMEROCARTAO,SEXO,LINE,NAME,URBS_CODE,STOP_ID,LAT,LON
45746,000,00010,25/11/76,"09/05/17 04:47:52,000000",OPER S/LINHA,2337319,M,000,TERMINAL SÍTIO CERCADO,00010,14492,-25.534235,-49.267368
46072,000,00010,22/09/66,"09/05/17 04:47:59,000000",OPER S/LINHA,3563770,M,000,TERMINAL SÍTIO CERCADO,00010,14492,-25.534235,-49.267368
46180,000,00010,28/06/56,"09/05/17 04:48:16,000000",OPER S/LINHA,1799518,M,000,TERMINAL SÍTIO CERCADO,00010,14492,-25.534235,-49.267368
96005,000,00012,29/03/77,"09/05/17 04:51:31,000000",OPER S/LINHA,3633298,M,000,TERMINAL SÍTIO CERCADO,00012,14492,-25.534235,-49.267368
81008,000,09003,19/11/52,"09/05/17 04:52:03,000000",OPER S/LINHA,1503781,F,000,TERMINAL FAZENDINHA,09003,14512,-25.477397,-49.326660
96002,000,00012,10/05/62,"09/05/17 04:52:04,000000",OPER S/LINHA,3316674,F,000,TERMINAL SÍTIO CERCADO,00012,14492,-25.534235,-49.267368
96417,000,00012,12/01/78,"09/05/17 04:52:22,000000",OPER S/LINHA,3702630,F,000,TERMINAL SÍTIO CERCADO,00012,14492,-25.534235,-49.267368
30401,000,03001,27/11/58,"09/05/17 04:53:01,000000",OPER S/LINHA,2441463,M,000,TERMINAL PINHEIRINHO,03001,14494,-25.512983,-49.294505
96358,000,00012,17/07/68,"09/05/17 04:53:15,000000",OPER S/LINHA,3325352,M,000,TERMINAL SÍTIO CERCADO,00012,14492,-25.534235,-49.267368
95563,000,00012,16/07/94,"09/05/17 04:53:58,000000",OPER S/LINHA,3727031,M,000,TERMINAL SÍTIO CERCADO,00012,14492,-25.534235,-49.267368


In [26]:
num_matched_stations = len(matched_000_stations)

In [27]:
print num_matched_stations, boardings_000, num_matched_stations/float(boardings_000)

122864 134629 0.912611695846


### Find Candidate bus routes for each boarding event

#### Read GPS data for the same day

In [42]:
gps_data = pd.read_csv('/local/tarciso/masters/data/bus_trips/buste-v3a/may_jun_jul/single-csvs/2017_05_09_veiculos.csv', dtype = {'route': str}, na_values='-')

In [43]:
gps_data.head()

,route,tripNum,shapeId,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,busCode,gpsPointId,gpsLat,gpsLon,distanceToShapePoint,timestamp,stopPointId,problem,birthdate,cardTimestamp,lineName,cardNum,gender
0,500,2.0,4130,6464140,-25.438837,-49.268129,9735.966,GE718,NaN,-25.438738,-49.268180,12.105323,06:53:44,25515,NO_PROBLEM,NaN,NaN,NaN,NaN,NaN
1,500,2.0,4130,6464167,-25.433790,-49.270262,10543.422,GE718,NaN,-25.433916,-49.270425,21.556105,06:58:25,27560,NO_PROBLEM,NaN,NaN,NaN,NaN,NaN
2,500,5.0,4130,6463793,-25.516848,-49.230057,0.000,GE718,NaN,-25.516885,-49.229913,15.014615,07:22:26,27537,NO_PROBLEM,NaN,NaN,NaN,NaN,NaN
3,500,1.0,4130,6463875,-25.501550,-49.237590,2129.862,GE718,NaN,NaN,NaN,NaN,07:26:47,27472,BETWEEN,NaN,NaN,NaN,NaN,NaN
4,500,1.0,4130,6463945,-25.481825,-49.246977,4513.260,GE718,NaN,NaN,NaN,NaN,07:28:04,27551,BETWEEN,NaN,NaN,NaN,NaN,NaN


In [58]:
gps_data.dtypes

route                     object
tripNum                  float64
shapeId                    int64
shapeSequence              int64
shapeLat                 float64
shapeLon                 float64
distanceTraveledShape    float64
busCode                   object
gpsPointId               float64
gpsLat                   float64
gpsLon                   float64
distanceToShapePoint     float64
timestamp                 object
stopPointId                int64
problem                   object
birthdate                 object
cardTimestamp             object
lineName                  object
cardNum                  float64
gender                    object
dtype: object

In [45]:
len(gps_data)

1303692

### Match BUSTE GPS entries with their respective parent stop codes

#### Read GTFS stops file

In [92]:
stops_df = pd.read_csv('/local/tarciso/data/gtfs/curitiba/stops.txt')

In [93]:
stops_df.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_timezone,wheelchair_boarding
0,70,104505,Terminal Campina do Siqueira - 303 - Centenári...,Terminal Campina do Siqueira - Campo Comprido,-25.435724,-49.306998,NaN,NaN,0,14506.0,NaN,NaN
1,270,104905,Terminal Carmo - 030 - Interbairros III,Terminal Carmo - 030 - Interbairros III (Senti...,-25.501341,-49.237597,NaN,NaN,0,14485.0,NaN,NaN
2,276,105606,Terminal Oficinas - 030 - Interbairros III,Terminal Oficinas - 030 - Interbairros III (Se...,-25.451550,-49.214917,NaN,NaN,0,14481.0,NaN,NaN
3,299,105603,Terminal Oficinas - 030 - Interbairros III,Terminal Oficinas - 030 - Interbairros III (Se...,-25.451665,-49.215086,NaN,NaN,0,14481.0,NaN,NaN
4,308,104907,Terminal Carmo - 030 - Interbairros III,Terminal Carmo - 030 - Interbairros III (Senti...,-25.501311,-49.237825,NaN,NaN,0,14485.0,NaN,NaN


In [94]:
stops_parent_stations = stops_df[['stop_id','parent_station']]

In [95]:
gps_stops_with_parent_stations = gps_data.merge(stops_parent_stations, left_on='stopPointId', right_on='stop_id', how='inner')

In [96]:
gps_stops_with_parent_stations.head()

,route,tripNum,shapeId,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,busCode,gpsPointId,gpsLat,...,timestamp,stopPointId,problem,birthdate,cardTimestamp,lineName,cardNum,gender,stop_id,parent_station
0,500,2.0,4130,6464140,-25.438837,-49.268129,9735.966,GE718,NaN,-25.438738,...,06:53:44,25515,NO_PROBLEM,NaN,NaN,NaN,NaN,NaN,25515,NaN
1,500,4.0,4130,6464140,-25.438837,-49.268129,9735.966,GE718,NaN,-25.438743,...,18:26:05,25515,TRIP_PROBLEM,NaN,NaN,NaN,NaN,NaN,25515,NaN
2,500,3.0,4130,6464140,-25.438837,-49.268129,9735.966,GE718,NaN,NaN,...,07:40:50,25515,BETWEEN,NaN,NaN,NaN,NaN,NaN,25515,NaN
3,500,5.0,4130,6464140,-25.438837,-49.268129,9735.966,GE718,NaN,NaN,...,08:31:53,25515,BETWEEN,NaN,NaN,NaN,NaN,NaN,25515,NaN
4,550,1.0,2000,4985410,-25.438978,-49.268149,10308.128,NaN,NaN,NaN,...,NaN,25515,BETWEEN,NaN,NaN,NaN,NaN,NaN,25515,NaN


#### Finding buses for the following boarding entry:

In [97]:
matched_000_stations[:1]

,CODLINHA,CODVEICULO,DATANASCIMENTO,DATAUTILIZACAO,NOMELINHA,NUMEROCARTAO,SEXO,LINE,NAME,URBS_CODE,STOP_ID,LAT,LON
45746,000,00010,25/11/76,"09/05/17 04:47:52,000000",OPER S/LINHA,2337319,M,000,TERMINAL SÍTIO CERCADO,00010,14492,-25.534235,-49.267368


In [98]:
type(gps_stops_with_parent_stations['stopPointId'][0])

numpy.int64

In [101]:
gps_stops_with_parent_stations.query('parent_station==14492')#.sort_values(by='timestamp')

,route,tripNum,shapeId,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,busCode,gpsPointId,gpsLat,...,timestamp,stopPointId,problem,birthdate,cardTimestamp,lineName,cardNum,gender,stop_id,parent_station
66810,540,3.0,1985,6328328,-25.534398,-49.267898,4568.548,KA691,NaN,NaN,...,12:23:06,5346,BETWEEN,08/02/86,05:32:54,BOQUEIRÃO/MONDELEZ,3789900.0,F,5346,14492.0
66811,540,3.0,1985,6328328,-25.534398,-49.267898,4568.548,KA691,NaN,NaN,...,12:23:06,5346,BETWEEN,22/06/68,05:23:31,BOQUEIRÃO/MONDELEZ,1816996.0,M,5346,14492.0
66812,540,3.0,1985,6328328,-25.534398,-49.267898,4568.548,KA691,NaN,NaN,...,12:23:06,5346,BETWEEN,24/02/93,05:36:04,BOQUEIRÃO/MONDELEZ,3762622.0,M,5346,14492.0
66813,540,3.0,1985,6328328,-25.534398,-49.267898,4568.548,KA691,NaN,NaN,...,12:23:06,5346,BETWEEN,14/08/79,05:21:41,BOQUEIRÃO/MONDELEZ,3692521.0,F,5346,14492.0
66814,540,2.0,1985,6328328,-25.534398,-49.267898,4568.548,KA691,NaN,-25.534263,...,06:57:20,5346,TRIP_PROBLEM,NaN,NaN,NaN,NaN,NaN,5346,14492.0
66815,540,1.0,1985,6328328,-25.534398,-49.267898,4568.548,KA691,NaN,-25.534408,...,05:31:04,5346,NO_PROBLEM,NaN,NaN,NaN,NaN,NaN,5346,14492.0
66816,540,8.0,1985,6328328,-25.534398,-49.267898,4568.548,KA691,NaN,-25.534401,...,13:40:15,5346,NO_PROBLEM,NaN,NaN,NaN,NaN,NaN,5346,14492.0
66817,540,5.0,1985,6328328,-25.534398,-49.267898,4568.548,KA691,NaN,NaN,...,09:36:41,5346,BETWEEN,NaN,NaN,NaN,NaN,NaN,5346,14492.0
66818,545,1.0,1991,6029464,-25.534431,-49.268030,4604.271,KA695,NaN,NaN,...,19:06:56,5346,BETWEEN,09/11/64,19:07:11,OP. CONTIGENCIA,2624114.0,M,5346,14492.0
66819,545,2.0,1991,6029464,-25.534431,-49.268030,4604.271,KA695,NaN,-25.534341,...,17:41:40,5346,NO_PROBLEM,NaN,NaN,NaN,NaN,NaN,5346,14492.0


In [84]:
gps_stops = gps_data[['stopPointId']].drop_duplicates()

In [90]:
gps_stops[gps_stops['stopPointId'] == 14487]

,stopPointId


In [86]:
gps_stops.merge(line_000_terminals, left_on='stopPointId', right_on='STOP_ID', how='inner')

,stopPointId,LINE,NAME,URBS_CODE,STOP_ID,LAT,LON
